# Question 1: Sentiment Analysis on IMDb Dataset



## Setup and Configuration

This cell sets up the entire environment for the experiment:

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import os
from datasets import load_dataset
from tqdm.auto import tqdm

# Set random seeds for reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Hyperparameters
EMB_DIM = 300
HID_DIM = 256
BATCH_SIZE = 32
OUTPUT_DIM = 1
DROPOUT = 0.5
LEARNING_RATE = 0.001
N_EPOCHS = 5
N_LAYERS = 2

## Data Pipeline (GloVe Path)

This cell implements the complete data preprocessing pipeline for GloVe-based models:

In [ ]:
# Import data processing utilities
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import re

# Load IMDB dataset
dataset = load_dataset("imdb")

# Split training data into train and validation sets
train_val_split = dataset['train'].train_test_split(test_size=0.1, seed=SEED)
dataset['train'] = train_val_split['train']
dataset['validation'] = train_val_split['test']

def clean_text(text):
    """Cleans text by lowercasing and expanding contractions."""
    text = text.lower()
    text = text.replace("<br />", " ").replace("<br>", " ")

    contractions = {
        "n't": " not",
        "'s": " is",
        "'m": " am",
        "'ll": " will",
        "'ve": " have",
        "'re": " are",
        "'d": " would",
    }

    for c, full in contractions.items():
        text = text.replace(c, full)

    return text

def glove_tokenizer(text):
    """Tokenizes text for GloVe embedding compatibility."""
    text = clean_text(text)
    tokens = re.findall(r"[a-z]+", text)
    return tokens

# Build vocabulary from training data
print("Building vocabulary...")
counter = Counter()

for text in tqdm(dataset['train']['text'], desc="Tokenizing Train Data"):
    counter.update(glove_tokenizer(text))

# Special tokens
PAD_IDX = 0
UNK_IDX = 1
vocab = {'<pad>': PAD_IDX, '<unk>': UNK_IDX}

# Limit vocabulary size
MAX_VOCAB_SIZE = 25_000

for word, freq in counter.most_common(MAX_VOCAB_SIZE):
    if word not in vocab:
        vocab[word] = len(vocab)

print(f"Vocab size: {len(vocab)}")

def text_pipeline(text):
    """Converts text to token indices."""
    return [vocab.get(token, UNK_IDX) for token in glove_tokenizer(text)]

MAX_LEN = 256

def collate_batch(batch):
    """Custom collate function for DataLoader with padding."""
    label_list, text_list, lengths = [], [], []

    for item in batch:
        label_list.append(float(item['label']))

        processed_list = text_pipeline(item['text'])

        # Truncate if exceeds max length
        if len(processed_list) > MAX_LEN:
            processed_list = processed_list[:MAX_LEN]

        processed_text = torch.tensor(processed_list, dtype=torch.long)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))

    # Pad sequences to same length
    text_list = pad_sequence(text_list, batch_first=True, padding_value=PAD_IDX)
    label_list = torch.tensor(label_list, dtype=torch.float).to(device)
    text_list = text_list.to(device)
    lengths = torch.tensor(lengths, dtype=torch.long).cpu()

    return text_list, label_list, lengths

# Create DataLoaders for GloVe path
train_loader_glove = DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)
val_loader_glove = DataLoader(dataset['validation'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_loader_glove = DataLoader(dataset['test'], batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

print("Data pipeline ready (with Advanced Cleaning).")

## GloVe Integration with Smart Caching

This cell implements an efficient GloVe embedding integration system:


In [ ]:
# Import utilities for GloVe download and processing
import urllib.request
import zipfile
import os
import numpy as np
import torch
import random

# GloVe configuration
CACHE_DIR = '.vector_cache'
GLOVE_ZIP_URL = 'http://nlp.stanford.edu/data/glove.6B.zip'
GLOVE_FILE = 'glove.6B.300d.txt'

if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

def download_and_extract_glove(cache_dir=CACHE_DIR):
    """Downloads and extracts GloVe vectors if not already cached."""
    target_path = os.path.join(cache_dir, GLOVE_FILE)
    zip_path = os.path.join(cache_dir, 'glove.6B.zip')

    if not os.path.exists(target_path):
        print(f"GloVe (300d) not found. Downloading original ZIP... ({GLOVE_ZIP_URL})")
        print("This may take 1-2 minutes depending on internet speed (862MB)...")

        urllib.request.urlretrieve(GLOVE_ZIP_URL, zip_path)

        print("Extracting only 300d vectors from the zip...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extract(GLOVE_FILE, cache_dir)

        print("Cleaning up (deleting Zip)...")
        os.remove(zip_path)
        print("Done.")
    else:
        print(f"GloVe vectors ready at {target_path}.")

    return target_path

def create_embedding_matrix(vocab, emb_dim=300):
    """Creates embedding matrix by matching vocabulary with GloVe vectors."""
    glove_path = download_and_extract_glove()

    print("Creating embedding matrix...")
    embeddings_index = {}

    # Load GloVe vectors
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
                if coefs.shape[0] == emb_dim:
                    embeddings_index[word] = coefs
            except:
                continue

    # Initialize embedding matrix
    embedding_matrix = np.zeros((len(vocab), emb_dim))
    hits = 0
    misses = 0
    missed_words = []

    # Fill embedding matrix with GloVe vectors
    for word, i in vocab.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            # Random initialization for OOV words
            if word != '<pad>':
                embedding_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))
            misses += 1
            missed_words.append(word)

    print(f"Embedding Matrix Ready. Shape: {embedding_matrix.shape}")
    print(f"Matched: {hits}, Unmatched: {misses}")

    if missed_words:
        sample_size = min(10, len(missed_words))
        print(f"Sample Unmatched: {random.sample(missed_words, sample_size)}")

    return torch.tensor(embedding_matrix, dtype=torch.float)

# Create embedding matrix if vocab exists
if 'vocab' in locals():
    embedding_matrix = create_embedding_matrix(vocab, EMB_DIM)
else:
    print("ERROR: 'vocab' not found. Please run Step 2.")

## RNN Classifier Architecture

Defines a modular RNN-based classifier supporting both bi-directional LSTM and GRU architectures:



In [ ]:
import torch
import torch.nn as nn

class RNNClassifier(nn.Module):
    """Bidirectional RNN classifier with configurable LSTM/GRU and pretrained embeddings."""
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 n_layers, bidirectional, dropout, pad_idx, rnn_type='LSTM', pretrained_embeddings=None):
        super().__init__()

        # Embedding layer: use pretrained if provided
        if pretrained_embeddings is not None:
            self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False, padding_idx=pad_idx)
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # RNN layer: LSTM or GRU
        self.rnn_type = rnn_type
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim,
                               hidden_dim,
                               num_layers=n_layers,
                               bidirectional=bidirectional,
                               dropout=dropout,
                               batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim,
                              hidden_dim,
                              num_layers=n_layers,
                              bidirectional=bidirectional,
                              dropout=dropout,
                              batch_first=True)
        else:
            raise ValueError("rnn_type must be 'LSTM' or 'GRU'")

        # Output layer: adjust for bidirectional
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        # Embed and apply dropout
        embedded = self.dropout(self.embedding(text))

        # Pack sequences for efficient RNN processing
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True, enforce_sorted=False)

        # Forward through RNN
        if self.rnn_type == 'LSTM':
            packed_output, (hidden, cell) = self.rnn(packed_embedded)
        else:
            packed_output, hidden = self.rnn(packed_embedded)

        # Concatenate final hidden states for bidirectional
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        return self.fc(hidden)

print("Model class UPDATED (GloVe supported).")

## Training Utilities

Defines essential training and evaluation functions:



In [ ]:
def binary_accuracy(preds, y):
    """Calculates binary classification accuracy."""
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion):
    """Trains the model for one epoch."""
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for text, label, lengths in iterator:
        optimizer.zero_grad()

        predictions = model(text, lengths).squeeze(1)

        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    """Evaluates the model on a dataset."""
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for text, label, lengths in iterator:
            predictions = model(text, lengths).squeeze(1)

            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

print("Train and Evaluate functions defined.")

## GloVe Experiment Loop: LSTM vs GRU

Trains and evaluates bidirectional LSTM and GRU models with GloVe embeddings:


### Output:
- Saves `best_model_LSTM.pt` and `best_model_GRU.pt`
- Prints comparison table showing test accuracy and training time for both models

In [ ]:
import time

# Training configuration
N_LAYERS = 2
models_to_train = ['LSTM', 'GRU']
results = {}
convergence_history = {}  # Track validation accuracy per epoch

print("="*60)
print(f"EXPERIMENT STARTING: {models_to_train}")
print(f"Hyperparameters: HID_DIM={HID_DIM}, Layers={N_LAYERS}, Dropout={DROPOUT}")
print("="*60)

# Train both LSTM and GRU models
for rnn_type in models_to_train:
    print(f"\n--- Training Starting: Bidirectional {rnn_type} + GloVe ---")

    # Initialize model with GloVe embeddings
    model = RNNClassifier(len(vocab), EMB_DIM, HID_DIM, OUTPUT_DIM,
                          N_LAYERS, bidirectional=True, dropout=DROPOUT,
                          pad_idx=PAD_IDX, rnn_type=rnn_type,
                          pretrained_embeddings=embedding_matrix).to(device)

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss().to(device)

    best_valid_acc = 0.0
    model_start_time = time.time()
    epoch_accuracies = []  # Store accuracies for convergence plot

    # Training loop
    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss, train_acc = train(model, train_loader_glove, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, val_loader_glove, criterion)
        
        epoch_accuracies.append(valid_acc)

        end_time = time.time()
        epoch_mins = int((end_time - start_time) / 60)
        epoch_secs = int((end_time - start_time) % 60)

        # Save best model based on validation accuracy
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            torch.save(model.state_dict(), f'best_model_{rnn_type}.pt')
            save_msg = "--> Model Saved!"
        else:
            save_msg = ""

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% {save_msg}')

    total_time = time.time() - model_start_time
    print(f"\n{rnn_type} Training Completed. Total Time: {int(total_time)} sec")
    
    # Store convergence history for visualization
    convergence_history[f'GloVe-{rnn_type}'] = epoch_accuracies

In [ ]:
# Import evaluation utilities
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import torch
import numpy as np
import os

print("="*80)
print("GLOVE MODEL EVALUATION")
print("="*80)


print("\n[1] Checking if required variables exist...")

required_vars = [
    'vocab', 'embedding_matrix', 'test_loader_glove', 'device',
    'EMB_DIM', 'HID_DIM', 'OUTPUT_DIM', 'N_LAYERS',
    'DROPOUT', 'PAD_IDX'
]

for var_name in required_vars:
    try:
        var = eval(var_name)
        if var_name == 'embedding_matrix':
            print(f" ✓ {var_name}: exists, shape = {var.shape}, dtype = {var.dtype}")
        elif var_name == 'vocab':
            print(f" ✓ {var_name}: exists, size = {len(var)}")
        else:
            print(f" ✓ {var_name}: exists")
    except NameError:
        print(f" ✗ {var_name}: NOT FOUND!")


print("\n[2] Checking if model files exist...")

for model_file in ['best_model_LSTM.pt', 'best_model_GRU.pt']:
    if os.path.exists(model_file):
        size_mb = os.path.getsize(model_file) / (1024**2)
        print(f" ✓ {model_file}: exists ({size_mb:.2f} MB)")
    else:
        print(f" ✗ {model_file}: NOT FOUND!")


print("\n[3] Loading LSTM model and checking architecture...")

try:
    model_lstm = RNNClassifier(
        len(vocab), EMB_DIM, HID_DIM, OUTPUT_DIM, N_LAYERS,
        bidirectional=True,
        dropout=DROPOUT,
        pad_idx=PAD_IDX,
        rnn_type='LSTM',
        pretrained_embeddings=embedding_matrix
    ).to(device)

    print(" ✓ Model created successfully")
    print(f" ✓ Embedding layer shape: {model_lstm.embedding.weight.shape}")
    print(f" ✓ Is embedding frozen? {not model_lstm.embedding.weight.requires_grad}")

    # Load saved weights
    model_lstm.load_state_dict(torch.load('best_model_LSTM.pt', weights_only=False))
    print(" ✓ Weights loaded successfully")

except Exception as e:
    print(f" ✗ Error creating/loading model: {e}")
    import traceback
    traceback.print_exc()

print("\n[4] Testing on first batch of test data...")

try:
    model_lstm.eval()

    for text, labels, lengths in test_loader_glove:
        print(f" → Batch shape: text={text.shape}, labels={labels.shape}, lengths={lengths.shape}")
        print(f" → Sample text indices: {text[0][:10]}")
        print(f" → Sample label: {labels[0].item()}")

        with torch.no_grad():
            predictions = model_lstm(text, lengths).squeeze(1)
            print(f" → Raw predictions (first 5): {predictions[:5]}")
            sigmoid_preds = torch.sigmoid(predictions)
            print(f" → After sigmoid (first 5): {sigmoid_preds[:5]}")
            rounded_preds = torch.round(sigmoid_preds)
            print(f" → After rounding (first 5): {rounded_preds[:5]}")

            unique_preds = torch.unique(rounded_preds)
            print(f" → Unique predictions in batch: {unique_preds}")

            if len(unique_preds) == 1:
                print(f" ⚠️ WARNING: All predictions are {unique_preds[0].item()}!")
        break

except Exception as e:
    print(f" ✗ Error during inference: {e}")
    import traceback
    traceback.print_exc()


print("\n[5] Running full test evaluation...")

glove_models = {
    'Bi-LSTM + GloVe': ('best_model_LSTM.pt', 'LSTM'),
    'Bi-GRU + GloVe': ('best_model_GRU.pt', 'GRU'),
}

for model_name, (model_path, rnn_type) in glove_models.items():
    print(f"\n--- Evaluating: {model_name} ---")

    try:
        # Initialize model with same architecture
        model = RNNClassifier(
            len(vocab), EMB_DIM, HID_DIM, OUTPUT_DIM, N_LAYERS,
            bidirectional=True,
            dropout=DROPOUT,
            pad_idx=PAD_IDX,
            rnn_type=rnn_type,
            pretrained_embeddings=embedding_matrix
        ).to(device)

        model.load_state_dict(torch.load(model_path, weights_only=False))
        model.eval()

        all_preds = []
        all_labels = []
        all_raw_logits = []

        # Evaluate on test set
        with torch.no_grad():
            for text, labels, lengths in tqdm(test_loader_glove, desc="Testing"):
                predictions = model(text, lengths).squeeze(1)

                all_raw_logits.extend(predictions.cpu().numpy())
                preds = torch.round(torch.sigmoid(predictions)).cpu().numpy()

                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        # Convert to numpy arrays
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_raw_logits = np.array(all_raw_logits)

        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        macro_f1 = f1_score(all_labels, all_preds, average='macro')

        print("\n📊 Results:")
        print(f" Test Accuracy: {accuracy*100:.2f}%")
        print(f" Test Macro-F1: {macro_f1:.4f}")

        print("\n📈 Prediction Distribution:")
        unique, counts = np.unique(all_preds, return_counts=True)
        for v, c in zip(unique, counts):
            print(f" Predicted {int(v)}: {c} samples ({c/len(all_preds)*100:.1f}%)")

        print("\n📈 True Label Distribution:")
        unique, counts = np.unique(all_labels, return_counts=True)
        for v, c in zip(unique, counts):
            print(f" True {int(v)}: {c} samples ({c/len(all_labels)*100:.1f}%)")

        print("\n📊 Raw Logits Statistics:")
        print(f" Min: {all_raw_logits.min():.4f}")
        print(f" Max: {all_raw_logits.max():.4f}")
        print(f" Mean: {all_raw_logits.mean():.4f}")
        print(f" Std: {all_raw_logits.std():.4f}")

    except Exception as e:
        print(f" ✗ Error: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*80)
print("DEBUG COMPLETE")
print("="*80)

## BERT Setup: Tokenizer and Data Loaders

Prepares the infrastructure for BERT-based experiments (distil-bert-uncased)



In [ ]:
# Import BERT tokenizer
from transformers import AutoTokenizer

print("="*60)
print("SECTION 2: PREPARING CONTEXTUAL EMBEDDINGS (DISTILBERT)")
print("="*60)

# Load DistilBERT tokenizer
MODEL_NAME = 'distilbert-base-uncased'
bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def collate_batch_bert(batch):
    """Custom collate function for BERT DataLoader."""
    text_list = [item['text'] for item in batch]
    label_list = [float(item['label']) for item in batch]

    # Tokenize with BERT tokenizer
    encodings = bert_tokenizer(text_list,
                                truncation=True,
                                padding=True,
                                max_length=256,
                                return_tensors='pt')

    labels = torch.tensor(label_list, dtype=torch.float).to(device)
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    return input_ids, attention_mask, labels

# Create DataLoaders for BERT path
BERT_BATCH_SIZE = 32

train_loader_bert = DataLoader(dataset['train'], batch_size=BERT_BATCH_SIZE, shuffle=True, collate_fn=collate_batch_bert)
val_loader_bert = DataLoader(dataset['validation'], batch_size=BERT_BATCH_SIZE, shuffle=False, collate_fn=collate_batch_bert)
test_loader_bert = DataLoader(dataset['test'], batch_size=BERT_BATCH_SIZE, shuffle=False, collate_fn=collate_batch_bert)

print(f"DistilBERT DataLoaders ready. Batch Size: {BERT_BATCH_SIZE}")

## BERT-RNN Classifier Architecture

Defines a hybrid architecture combining frozen BERT with trainable RNN:


In [ ]:
# Import BERT model
from transformers import AutoModel

class BERTRNNClassifier(nn.Module):
    """Hybrid classifier: frozen BERT encoder + trainable bidirectional RNN."""
    
    def __init__(self, rnn_type, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()

        # Load pretrained BERT and freeze parameters
        self.bert = AutoModel.from_pretrained('distilbert-base-uncased')

        for param in self.bert.parameters():
            param.requires_grad = False

        embedding_dim = self.bert.config.hidden_size

        # RNN layer on top of BERT
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers,
                               bidirectional=bidirectional, dropout=dropout, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=n_layers,
                              bidirectional=bidirectional, dropout=dropout, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        # Get BERT embeddings (frozen)
        with torch.no_grad():
            bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        embedded = self.dropout(bert_output.last_hidden_state)

        # Forward through RNN
        if isinstance(self.rnn, nn.LSTM):
            output, (hidden, cell) = self.rnn(embedded)
        else:
            output, hidden = self.rnn(embedded)

        # Concatenate bidirectional hidden states
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        return self.fc(hidden)

print("DistilBERT-RNN Architecture Defined.")

## BERT Experiment Loop: LSTM vs GRU

Trains and evaluates BERT-based models with LSTM and GRU heads:

### Output Format:
- Saves best models: `best_bert_LSTM.pt` and `best_bert_GRU.pt`
- Based on validation accuracy


In [ ]:
import time

# Training functions for BERT models
def train_bert(model, iterator, optimizer, criterion):
    """Trains BERT model for one epoch."""
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for input_ids, mask, label in iterator:
        optimizer.zero_grad()
        predictions = model(input_ids, mask).squeeze(1)
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate_bert(model, iterator, criterion):
    """Evaluates BERT model on a dataset."""
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for input_ids, mask, label in iterator:
            predictions = model(input_ids, mask).squeeze(1)
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Train both LSTM and GRU with BERT embeddings
bert_models = ['LSTM', 'GRU']

print(f"\nDistilBERT EXPERIMENT STARTING: {bert_models}")

for rnn_type in bert_models:
    print(f"\n--- Training Starting: DistilBERT + Bi-{rnn_type} ---")

    # Initialize model
    model = BERTRNNClassifier(rnn_type, HID_DIM, OUTPUT_DIM, N_LAYERS, True, DROPOUT).to(device)

    # Only optimize RNN parameters (BERT is frozen)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
    criterion = nn.BCEWithLogitsLoss().to(device)

    best_valid_acc = 0.0
    epoch_accuracies = []

    # Training loop
    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss, train_acc = train_bert(model, train_loader_bert, optimizer, criterion)
        valid_loss, valid_acc = evaluate_bert(model, val_loader_bert, criterion)
        
        epoch_accuracies.append(valid_acc)

        end_time = time.time()
        epoch_mins = int((end_time - start_time) / 60)
        epoch_secs = int((end_time - start_time) % 60)

        # Save best model
        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            torch.save(model.state_dict(), f'best_bert_{rnn_type}.pt')
            save_msg = "--> Saved"
        else:
            save_msg = ""

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% {save_msg}')
    
    # Store for convergence visualization
    convergence_history[f'BERT-{rnn_type}'] = epoch_accuracies

print("\nBERT Trainings Completed. Models saved.")

## Final Evaluation: Comprehensive Metrics

Performs rigorous evaluation of all four trained models on the test set:



In [ ]:
# Import evaluation and visualization libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
import torch
from tqdm.auto import tqdm

print("="*60)
print("EVALUATION AND PLOTTING OF ALL MODELS")
print("="*60)

# Storage for final results
final_metrics = {
    'Model': [],
    'Accuracy': [],
    'Macro-F1': []
}

# Define all models to evaluate
models_to_evaluate = {
    'Bi-LSTM + GloVe': ('best_model_LSTM.pt', RNNClassifier, 'LSTM', False),
    'Bi-GRU + GloVe':  ('best_model_GRU.pt',  RNNClassifier, 'GRU',  False),
    'DistilBERT + Bi-LSTM': ('best_bert_LSTM.pt', BERTRNNClassifier, 'LSTM', True),
    'DistilBERT + Bi-GRU':  ('best_bert_GRU.pt',  BERTRNNClassifier, 'GRU',  True),
}

# Evaluate each model
for model_name, (model_path, model_class, rnn_type, is_bert_model) in models_to_evaluate.items():
    print(f"Evaluating: {model_name}...")

    # Initialize model with appropriate architecture
    if is_bert_model:
        model = model_class(rnn_type, HID_DIM, OUTPUT_DIM, N_LAYERS, True, DROPOUT).to(device)
        current_loader = test_loader_bert
    else:
        model = model_class(len(vocab), EMB_DIM, HID_DIM, OUTPUT_DIM, 
                            N_LAYERS, bidirectional=True, dropout=DROPOUT, 
                            pad_idx=PAD_IDX, rnn_type=rnn_type, 
                            pretrained_embeddings=embedding_matrix).to(device)
        current_loader = test_loader_glove
    
    # Load saved weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    all_preds = []
    all_labels = []

    # Run inference on test set
    with torch.no_grad():
        for batch in current_loader:
            if is_bert_model:
                input_ids, attention_mask, labels = batch
                predictions = model(input_ids, attention_mask).squeeze(1)
            else:
                text, labels, lengths = batch
                predictions = model(text, lengths).squeeze(1)
            
            preds = torch.round(torch.sigmoid(predictions)).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Store results
    final_metrics['Model'].append(model_name)
    final_metrics['Accuracy'].append(acc)
    final_metrics['Macro-F1'].append(f1)

# Create results DataFrame
df_results = pd.DataFrame(final_metrics)
print("\n Output Table:")
print(df_results)

# Visualization: comparison bar plot
df_melted = df_results.melt(id_vars="Model", var_name="Metric", value_name="Score")

plt.figure(figsize=(10, 6))
ax = sns.barplot(x="Model", y="Score", hue="Metric", data=df_melted, palette="viridis")

# Add value labels on bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.4f', padding=3, fontsize=10, fontweight='bold')

plt.title('Model Performance Comparison (Auto-Generated)', fontsize=16, fontweight='bold')

# Set y-axis limits for better visualization
min_score = df_melted['Score'].min()
plt.ylim(min_score - 0.02, min_score + 0.05) 

plt.ylabel('Score', fontsize=12)
plt.xlabel('Architecture', fontsize=12)
plt.legend(loc='lower right', title='Metric')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('performance_comparison_auto.png', dpi=300)
plt.show()

## Convergence Efficiency Analysis

Visualizes how quickly each model reaches the accuracy threshold (0.87):




In [ ]:
# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set plot style
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 12})

# Threshold for convergence analysis
THRESHOLD = 0.87

plt.figure(figsize=(12, 6))

# Color palette: Blues for GloVe, Oranges for BERT
colors = {
    'GloVe-LSTM': '#1f77b4',  # Dark Blue
    'GloVe-GRU':  '#6baed6',  # Light Blue
    'BERT-LSTM':  '#ff7f0e',  # Dark Orange
    'BERT-GRU':   '#fdae6b'   # Light Orange
}

# Plot convergence curves for all models
for model_name, accuracies in convergence_history.items():
    epochs = range(1, len(accuracies) + 1)
    
    # Line plot with markers
    plt.plot(epochs, accuracies, label=model_name, color=colors.get(model_name, 'black'), linewidth=2.5, marker='o')
    
    # Mark first epoch where threshold is reached
    for i, acc in enumerate(accuracies):
        if acc >= THRESHOLD:
            plt.plot(i+1, acc, marker='*', color='red', markersize=15, zorder=10)
            plt.text(i+1, acc+0.005, f"{i+1}. Epoch", color='red', fontsize=9, fontweight='bold')
            break

# Threshold line
plt.axhline(y=THRESHOLD, color='r', linestyle='--', linewidth=2, alpha=0.7, label=f'Threshold ({THRESHOLD})')

plt.title('Convergence Efficiency: Validation Accuracy per Epoch', fontsize=16, fontweight='bold')
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Validation Accuracy', fontsize=14)
plt.ylim(0.80, 0.95) 
plt.legend(loc='lower right')
plt.tight_layout()

plt.savefig('convergence_efficiency.png', dpi=300)
plt.show()
print("Plot saved as 'convergence_efficiency.png'.")

## Latent Space Visualization: PCA & t-SNE

Visualizes and compares the learned representations of all four models:


In [ ]:
# Import dimensionality reduction and visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch
import torch.nn as nn

# Visualization parameters
N_SAMPLES = 1000
PERPLEXITY = 30
SEED = 1234

# Custom color palette: Blue for negative, Orange for positive
CUSTOM_PALETTE = ["#1f77b4", "#ff7f0e"]

print("="*60)
print("LATENT SPACE VISUALIZATION (BLUE/ORANGE)")
print("="*60)

# Create fresh DataLoaders for visualization
try:
    viz_loader_glove = DataLoader(dataset['test'], batch_size=64, shuffle=True, collate_fn=collate_batch)
    print("  GloVe Loader refreshed.")
except NameError:
    print("ERROR: 'collate_batch' not found. Run Step 2.")

try:
    viz_loader_bert = DataLoader(dataset['test'], batch_size=32, shuffle=True, collate_fn=collate_batch_bert)
    print("  BERT Loader refreshed.")
except NameError:
    print("ERROR: 'collate_batch_bert' not found. Run Cell 7.")

# Configuration for all models
viz_configs = [
    ("GloVe-LSTM", "best_model_LSTM.pt", "RNN", viz_loader_glove, False),
    ("GloVe-GRU",  "best_model_GRU.pt",  "RNN", viz_loader_glove, False),
    ("BERT-LSTM",  "best_bert_LSTM.pt",  "BERT", viz_loader_bert, True),
    ("BERT-GRU",   "best_bert_GRU.pt",   "BERT", viz_loader_bert, True)
]

def extract_features_robust(model, loader, is_bert, model_name):
    """Extracts latent representations from RNN hidden states."""
    model.eval()
    features_list = []
    labels_list = []
    total_count = 0

    with torch.no_grad():
        for i, batch in enumerate(loader):
            if total_count >= N_SAMPLES: break

            if is_bert:
                # BERT path: get hidden states from BERT then RNN
                inputs, mask, targets = batch
                bert_out = model.bert(inputs, mask)
                embedded = model.dropout(bert_out.last_hidden_state)
                if "LSTM" in model_name:
                    _, (hidden, _) = model.rnn(embedded)
                else:
                    _, hidden = model.rnn(embedded)
            else:
                # GloVe path: embed then RNN
                text, targets, lengths = batch
                embedded = model.dropout(model.embedding(text))
                packed_embedded = torch.nn.utils.rnn.pack_padded_sequence(
                    embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
                )
                if "LSTM" in model_name:
                    _, (hidden, _) = model.rnn(packed_embedded)
                else:
                    _, hidden = model.rnn(packed_embedded)

            # Concatenate bidirectional hidden states
            if model.rnn.bidirectional:
                latent_vec = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
            else:
                latent_vec = hidden[-1,:,:]

            features_list.append(latent_vec.cpu())
            labels_list.append(targets.cpu())
            total_count += len(targets)

    if len(features_list) > 0:
        features = torch.cat(features_list, dim=0)[:N_SAMPLES].numpy()
        labels = torch.cat(labels_list, dim=0)[:N_SAMPLES].numpy()
        print(f"  -> {model_name}: {len(features)} samples successfully extracted.")
        return features, labels
    else:
        print(f"  -> ERROR: No data could be extracted for {model_name}!")
        return np.array([]), np.array([])

# Create 2x4 subplot grid: PCA on top, t-SNE on bottom
fig, axes = plt.subplots(2, 4, figsize=(24, 12))
plt.subplots_adjust(hspace=0.3, wspace=0.3)

print("\nFetching and processing data...")

for idx, (name, path, m_type, loader, is_bert) in enumerate(viz_configs):

    try:
        # Load model
        if m_type == "RNN":
            model = RNNClassifier(len(vocab), EMB_DIM, HID_DIM, OUTPUT_DIM,
                                  N_LAYERS, True, DROPOUT, PAD_IDX,
                                  rnn_type=name.split('-')[1], pretrained_embeddings=None).to(device)
        else:
            model = BERTRNNClassifier(name.split('-')[1], HID_DIM, OUTPUT_DIM,
                                      N_LAYERS, True, DROPOUT).to(device)
        model.load_state_dict(torch.load(path))
    except Exception as e:
        print(f"Model Loading Error ({name}): {e}")
        continue

    # Extract features
    feats, y = extract_features_robust(model, loader, is_bert, name)

    if len(feats) == 0: continue

    # Standardize features
    scaler = StandardScaler()
    feats_norm = scaler.fit_transform(feats)

    # PCA reduction
    pca = PCA(n_components=2, random_state=SEED)
    pca_res = pca.fit_transform(feats_norm)

    # Plot PCA
    sns.scatterplot(x=pca_res[:,0], y=pca_res[:,1], hue=y, palette=CUSTOM_PALETTE,
                    s=40, alpha=0.7, ax=axes[0, idx], legend=False, linewidth=0)
    axes[0, idx].set_title(f"{name} - PCA")
    axes[0, idx].set_xticks([])
    axes[0, idx].set_yticks([])
    if idx == 0: axes[0, idx].set_ylabel("PCA", fontsize=12, fontweight='bold')

    # t-SNE reduction
    tsne = TSNE(n_components=2, random_state=SEED, perplexity=PERPLEXITY, n_iter=1000)
    tsne_res = tsne.fit_transform(feats_norm)

    # Plot t-SNE
    sns.scatterplot(x=tsne_res[:,0], y=tsne_res[:,1], hue=y, palette=CUSTOM_PALETTE,
                    s=40, alpha=0.7, ax=axes[1, idx], legend=(idx==3), linewidth=0)
    axes[1, idx].set_title(f"{name} - t-SNE")
    axes[1, idx].set_xticks([])
    axes[1, idx].set_yticks([])

# Add legend to last subplot
handles, _ = axes[1, 3].get_legend_handles_labels()
if handles:
    axes[1, 3].legend(handles, ["Negative (Blue)", "Positive (Orange)"], title="Sentiment", loc='upper right')

plt.suptitle("Latent Space: GloVe vs BERT (Blue/Orange Separation)", fontsize=18, y=0.95)
plt.savefig("latent_space_blue_orange.png", dpi=300, bbox_inches='tight')
plt.show()

print("Plot saved as 'latent_space_blue_orange.png'.")